# 🔧 Notebook 02 : Transformation des Données

**Auteur** : Oumaro Titans DJIGUIMDE  
**Date** : Février 2026  
**Objectif** : Transformer et enrichir le flux de transactions pour l'analyse

---

## 🎯 Objectifs de ce notebook

1. Charger le flux de transactions
2. Nettoyer et valider les données
3. Extraire des features temporelles
4. Enrichir avec des métriques dérivées
5. Créer des agrégations
6. Préparer les données pour l'analyse

## 📦 Importation des bibliothèques

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print("✅ Bibliothèques importées avec succès")

## 1️⃣ Chargement du flux

In [ ]:
print("="*70)
print("📂 CHARGEMENT DU FLUX")
print("="*70)

# Chargement
df = pd.read_csv('../data/transactions_stream.csv')

print(f"\n✅ Flux chargé")
print(f"   • Transactions : {len(df):,}")
print(f"   • Colonnes : {len(df.columns)}")
print("="*70)

In [ ]:
# Aperçu
print("\n👀 Aperçu des données :")
df.head(10)

In [ ]:
# Informations
print("\n📋 Informations :")
df.info()

## 2️⃣ Nettoyage et validation

In [ ]:
print("🧹 NETTOYAGE DES DONNÉES")
print("="*70)

# Conversion timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'])
print("✅ Timestamp converti en datetime")

# Tri par timestamp
df = df.sort_values('timestamp').reset_index(drop=True)
print("✅ Données triées par ordre chronologique")

# Vérification valeurs manquantes
missing = df.isnull().sum()
print(f"\n📊 Valeurs manquantes : {missing.sum()}")
if missing.sum() > 0:
    print(missing[missing > 0])

# Suppression doublons (si présents)
doublons_avant = df.duplicated().sum()
df = df.drop_duplicates()
doublons_apres = df.duplicated().sum()
print(f"\n🔄 Doublons supprimés : {doublons_avant}")

print(f"\n✅ Données nettoyées : {len(df):,} transactions")
print("="*70)

## 3️⃣ Extraction de features temporelles

In [ ]:
print("⏰ EXTRACTION DE FEATURES TEMPORELLES")
print("="*70)

# Date et heure
df['date'] = df['timestamp'].dt.date
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek  # 0=Lundi, 6=Dimanche
df['day_name'] = df['timestamp'].dt.day_name()
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

print("✅ Features extraites :")
print("   • date (jour)")
print("   • hour (heure 0-23)")
print("   • day_of_week (0-6)")
print("   • day_name (nom du jour)")
print("   • is_weekend (0/1)")

# Période de la journée
def get_period(hour):
    if 6 <= hour < 12:
        return 'Matin'
    elif 12 <= hour < 18:
        return 'Après-midi'
    elif 18 <= hour < 22:
        return 'Soir'
    else:
        return 'Nuit'

df['period'] = df['hour'].apply(get_period)
print("   • period (Matin/Après-midi/Soir/Nuit)")

print(f"\n✅ {len(df.columns)} colonnes totales")
print("="*70)

In [ ]:
# Aperçu des nouvelles features
print("\n👀 Aperçu avec features temporelles :")
df[['timestamp', 'date', 'hour', 'day_name', 'period', 'is_weekend', 'amount']].head(10)

## 4️⃣ Enrichissement avec métriques dérivées

In [ ]:
print("➕ CRÉATION DE MÉTRIQUES DÉRIVÉES")
print("="*70)

# Catégorie de montant
df['amount_category'] = pd.cut(
    df['amount'],
    bins=[0, 10000, 50000, 100000, float('inf')],
    labels=['Faible', 'Moyen', 'Élevé', 'Très élevé']
)
print("✅ Catégorie de montant créée")

# Log du montant (pour normalisation)
df['amount_log'] = np.log1p(df['amount'])
print("✅ Log du montant créé")

# Rang de la transaction par jour
df['transaction_rank'] = df.groupby('date').cumcount() + 1
print("✅ Rang de transaction par jour créé")

print("\n" + "="*70)

In [ ]:
# Aperçu
print("\n👀 Aperçu avec métriques dérivées :")
df[['timestamp', 'amount', 'amount_category', 'amount_log', 'transaction_rank']].head(10)

## 5️⃣ Agrégations

In [ ]:
print("📊 AGRÉGATIONS PAR DIMENSIONS")
print("="*70)

# Par ville
agg_ville = df.groupby('city').agg({
    'transaction_id': 'count',
    'amount': ['sum', 'mean', 'median', 'std']
})
agg_ville.columns = ['Nb_transactions', 'Volume_total', 'Montant_moyen', 'Montant_median', 'Ecart_type']
agg_ville = agg_ville.sort_values('Volume_total', ascending=False)

print("\n🏙️  Agrégation par VILLE :")
print(agg_ville)
print("\n" + "="*70)

In [ ]:
# Par type de transaction
agg_type = df.groupby('transaction_type').agg({
    'transaction_id': 'count',
    'amount': ['sum', 'mean', 'median']
})
agg_type.columns = ['Nb_transactions', 'Volume_total', 'Montant_moyen', 'Montant_median']
agg_type = agg_type.sort_values('Volume_total', ascending=False)

print("💳 Agrégation par TYPE :")
print(agg_type)
print("\n" + "="*70)

In [ ]:
# Par heure
agg_hour = df.groupby('hour').agg({
    'transaction_id': 'count',
    'amount': 'sum'
})
agg_hour.columns = ['Nb_transactions', 'Volume_total']

print("⏰ Agrégation par HEURE :")
print(agg_hour)
print("\n" + "="*70)

In [ ]:
# Par jour
agg_day = df.groupby('date').agg({
    'transaction_id': 'count',
    'amount': 'sum'
})
agg_day.columns = ['Nb_transactions', 'Volume_total']

print("📅 Agrégation par JOUR :")
print(agg_day.head(10))
print("\n" + "="*70)

## 6️⃣ Visualisation des transformations

In [ ]:
# Activité par heure
plt.figure(figsize=(14, 6))
agg_hour['Nb_transactions'].plot(kind='bar', color='skyblue', alpha=0.8)
plt.title('Nombre de Transactions par Heure', fontsize=16, fontweight='bold')
plt.xlabel('Heure', fontsize=12)
plt.ylabel('Nombre de transactions', fontsize=12)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Évolution temporelle
plt.figure(figsize=(14, 6))
agg_day['Volume_total'].plot(kind='line', marker='o', color='green', linewidth=2, markersize=6)
plt.title('Évolution du Volume Transactionnel', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Volume (FCFA)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Distribution par période et jour
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Par période
df['period'].value_counts().plot(kind='bar', ax=axes[0], color='coral', alpha=0.8)
axes[0].set_title('Transactions par Période', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Période')
axes[0].set_ylabel('Nombre de transactions')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3, axis='y')

# Par jour de la semaine
df['day_name'].value_counts().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
).plot(kind='bar', ax=axes[1], color='lightgreen', alpha=0.8)
axes[1].set_title('Transactions par Jour de la Semaine', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Jour')
axes[1].set_ylabel('Nombre de transactions')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 7️⃣ Sauvegarde des données transformées

In [ ]:
print("💾 SAUVEGARDE DES DONNÉES TRANSFORMÉES")
print("="*70)

# Sauvegarder le dataset enrichi
output_path = '../data/transactions_transformed.csv'
df.to_csv(output_path, index=False, encoding='utf-8')

print(f"\n✅ Dataset transformé sauvegardé : {output_path}")
print(f"📊 Lignes : {len(df):,}")
print(f"📋 Colonnes : {len(df.columns)}")
print(f"💾 Taille : {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Sauvegarder les agrégations
agg_ville.to_csv('../data/agg_ville.csv')
agg_type.to_csv('../data/agg_type.csv')
agg_hour.to_csv('../data/agg_hour.csv')
agg_day.to_csv('../data/agg_day.csv')

print("\n✅ Agrégations sauvegardées :")
print("   • agg_ville.csv")
print("   • agg_type.csv")
print("   • agg_hour.csv")
print("   • agg_day.csv")

print("\n" + "="*70)

## 📌 Résumé des transformations

### ✅ Transformations appliquées

1. **Nettoyage** :
   - ✅ Conversion des timestamps
   - ✅ Tri chronologique
   - ✅ Suppression des doublons

2. **Features temporelles** :
   - ✅ Date, heure, jour de la semaine
   - ✅ Période de la journée
   - ✅ Flag weekend

3. **Métriques dérivées** :
   - ✅ Catégorie de montant
   - ✅ Log du montant
   - ✅ Rang de transaction

4. **Agrégations** :
   - ✅ Par ville, type, heure, jour
   - ✅ Volumes et moyennes

### 📊 Dataset enrichi

- **Colonnes initiales** : 8
- **Colonnes finales** : 15+
- **Nouvelles features** : 7+
- **Agrégations créées** : 4 fichiers

### 🚀 Prochaine étape

**Notebook 03** : Analyse et Dashboard
- KPIs business
- Visualisations avancées
- Insights métier
- Recommandations